# PPD: MPI e programação com passagem de mensagem

Hélio - DC/UFSCar - 2023

# Sobre MPI

[MPI](https://www.mpi-forum.org) é uma biblioteca para programação paralela distribuída, comumente usada para a criação de aplicações que se espalham por diferentes computadores interligados em rede.

Os recursos de MPI permitem não só **iniciar** os processos em nós distintos, mas também oferecem formas de **identificar** logicamente os processos, o que simplifica muito as comunicações entre eles. Além disso, a biblioteca trata do **empacotamento** e **desempacotamento** dos dados em *buffers* para as transmissões, o que evita que o programador tenha que preocupar-se com conversões de formato e com a preservação dos significados dos dados transmitidos.

O modelo de aplicação é comumente **SPMD**, em que um nó (processo) mestre executa funções de coordenação e controle e os demais nós são trabalhadores, que usualmente replicam suas atividades sobre partes disjuntas dos dados.

# Estrutura dos programas

Para usar a estrutura MPI, é preciso fazer uma chamada à função **MPI_Init**, o que deve ocorrer antes de qualquer outra chamada a funções da biblioteca.

Tipicamente, os mesmos parâmetros que foram recebidos pela função *main* (argc e argv) são repassados à função de ativação da biblioteca.

Cabe à função [MPI_Init](https://www.open-mpi.org/doc/v3.1/man3/MPI_Init.3.php) executar as inicializações locais ao programa, como eventuais alocações de *buffers* internos, além de realizar comunicações com os demais nós lançados na ativação da aplicação. Tudo isso, contudo, é feito de forma transparente para a aplicação que invoca a chamada.

Ao final do programa, é preciso executar a função [MPI_Finalize](https://www.open-mpi.org/doc/v4.0/man3/MPI_Finalize.3.php), que encerra as operações da biblioteca.




# Grupo de processos

Uma aplicação MPI é comumente composta por um **grupo de processos**, epecificados na ativação do programa.

Vários sub-grupos podem ser formados, de acordo com a lógica da aplicação, mas há ao menos um grupo padrão, chamado **MPI_COMM_WORLD**. Esse grupo inclui todos os processos da aplicação e, dentro desse grupo, cada processo é identificado por um **número lógico**, chamado ***rank***, que varia de 0 a N-1. O índice 0 é atribuído ao processo que iniciou a aplicação.

Um aspecto interessante desse conceito de grupos e da identificação lógica dos processos é que a aplicação não precisa preocupar-se com os endereços (IP, e.g) dos nós em que os processos foram iniciados, ou mesmo com detalhes do protocolo da camada de transporte e da identificação dos números de portas associdaos a *sockets*.

Como veremos, nas operações de transmissão da API MPI, basta identificar os processos de acordo com seus números lógicos, cabendo à biblioteca cuidar de todos os detalhes para que as comunicações ocorram entre os processos apropriados.

Tremenda mão na roda, não é?!

Uma vez compilado, um programa MPI pode ser ativado com números variados de processos; ou seja, pode-se experimentar usar mais ou menos processos em cada execução, sem que seja preciso recompilar o programa.

Para que a aplicação saiba em tempo de execução quantos processos estão sendo usados nesta execução, a biblioteca oferece funções que retornam este número. Há também uma função para que cada processo saiba qual é seu índice lógico dentro do grupo MPI_COMM_WORLD (e de qualquer outro sub-grupo).

Esses serviços são providos pelas funções [MPI_Comm_size](https://www.open-mpi.org/doc/v3.1/man3/MPI_Comm_size.3.php)() e [MPI_Comm_rank](https://www.open-mpi.org/doc/v4.0/man3/MPI_Comm_rank.3.php)().

Vejamos um exemplo de código MPI a seguir.


In [ ]:
%%writefile m1.c

#include <mpi.h>
#include <stdio.h>

int
main( int argc, char *argv[])
{
	int numtasks, rank, status, namelen;
	char processor_name[MPI_MAX_PROCESSOR_NAME];

	// função obrigatória, usada para inicialização das atividades relacionadas a MPI
	status = MPI_Init(&argc,&argv);

	if (status != MPI_SUCCESS) {
		printf ("Erro em MPI_Init. Terminando...\n");
		MPI_Abort(MPI_COMM_WORLD, status);
	}

  // obtém o número de processos sendo usados nesta execução
	MPI_Comm_size(MPI_COMM_WORLD,&numtasks);

  // obtém o rank deste processo em relação aos processos da aplicação
	MPI_Comm_rank(MPI_COMM_WORLD,&rank);

	// int MPI_Get_processor_name(char *name, int *resultlen);
	// Returns the name of the processor on which it was called.
	MPI_Get_processor_name(processor_name, &namelen);

	printf("Processo %d de %d em %s\n", rank,numtasks,processor_name);

	printf("rank (%d): ",rank);
	for(int i=0; i< argc; i++)
		printf("%d: %s    ",i,argv[i]);
	printf("\n"); fflush(stdout);

	MPI_Finalize();

	return(0);
}

Overwriting m1.c


# Compilação de programas MPI

As extensões providas para uso de MPI incluem suporte para as linguagens C/C++ e Fortran.

Tomando como referência a programação em C, a compilação de programas envolve os seguintes aspectos:

* inclusão do cabeçalho <mpi.h> contendo as definições de tipos e os protótipos das funções da biblioteca;
* especificação da localização desta bibliteca para a compilação do programa;
* especificação da localização das bibliotecas com os códigos compilados das funções MPI para a fase de "ligação" (*link*) do programa;
* indicação para ligação (*link*) do código das biblioteas na fase de geração de código, ou ajuste para carregamento da bibliteca em tempo de execução, no caso de ligação dinâmica.

Para simplificar a compilação dos programas, há algunsn *scripts* utilitários que são instalados junto com a aplicação, como o comando [mpicc](https://www.open-mpi.org/doc/v4.0/man1/mpicc.1.php).

De maneira geral, usa-se este comando com a mesma sintaxe das chamadas a gcc.

```
$ mpicc prog.c -o prog
```

Vejamos como compilar o arquivo de programa apresentado anteriormente (m1.c).

In [ ]:
# Vamos verificar se MPI está instalado e se o utlitário mpicc está acessível
! whereis mpicc

# Compilemos o programa
! mpicc m1.c -o m1

! ls -l m1*

mpicc: /usr/bin/mpicc /usr/share/man/man1/mpicc.1.gz
-rwxr-xr-x 1 root root 16320 Dec  6 20:05 m1
-rw-r--r-- 1 root root   898 Dec  6 20:05 m1.c


# Ativação de programas MPI

A ativação de um programa mpi é comumente feita com os comandos **[mpirun](https://www.open-mpi.org/doc/current/man1/mpirun.1.php)**, ou ***mpiexec*** ou ***orterun***, para a implementação **OpenMPI**.

A ativação de programas pode ocorrer nos modelos **SPMD** (*single program multiple data*) e **MPMD** (*multiple program multiple data*).

* **SPMD**: o mesmo programa (arquivo executável) é ativado em todos os nós. Cabe à lógica do programa diferenciar cada instância e fazer com que trechos de código específicos, como uma função mestre e outra trabalhador, dentro do mesmo programa, sejam executadas nos nós. \

      $ mpirun [opções] programa


* **MPMD**: programas distintos podem ser especificados para nós ou grupoos de nós. \

      $ mpirun [ opções globais ] [ opções locais 1] ... : [opções locais 2]  
      // : é usado para separar cada programa e hosts que o executarão
      // O parâmetro -np, ou -n, indica o número de processos que serão usados.

Já os *hosts* a serem utilizados podem ser especificados individualmente na linha de comando, ou podem estar listados num arquivo texto especificado.

* Especificação dos hosts:
      $ mpirun -n 3 -host h1,h2,h3  prog ...

* Especificação de arquivo de *hosts*:
    
      $ mpirun -hostfile arq_hosts  prog ...
  ***ou***
      $ mpirun --machinefile arq_hosts prog ...

Exemplos:
```
$ mpirun prog
$ mpirun -np 4 prog                                 // executa o programa com 4 processos
$ mpirun -np 4 -host serv1,serv2,serv3,serv3 prog   // distribui 4 instâncias do programa, sendo 2 em serv3
$ mpirun -np 2 prog1 : -np 4 prog2                  // cria programa com 2 instâncias de prog1 e 4 de prog2
$ mpirun -np 4 -hostfile hosts prog                 // especifica o arquivo com a configuração dos hosts
$ mpirun -n 10 -host serv1:2,serv2:4,serv3:4
```

Uma vez indicados os *hosts*, a atribuição de processos a esses nós pode ocorrer segundo 2 políticas: ***by slot*** e ***by node***.

* ***By slot***: política padrão, na qual MPI atribui processos aos nós até completarem-se os *slots* disponíveis.
* ***By node***: selecionada com o parâmetro *--bynode* na ativação do programa, essa política faz com que MPI atribua um processo a cada nó de cada vez, de forma circular entre os nós com *slots* disponíveis.


O exemplo a seguir gera um arquivo de hosts (*hostfile*). Observe que diferentes opções são comentadas, mas apenas a indicação de que o host local está disponível com 4 *slots* será considerada. As demais opções estão comentadas no arquivo, usando o caractere # no início da linha.

Vale saber que, com TCP/IP, o nome DNS [*localhost*](https://en.wikipedia.org/wiki/Localhost) sempre se refere ao próprio computador e está associado ao endereço 127.0.0.1, que é um endereço reservado e associado à interface *loopback*, local a todo sistema. Ou seja, qualquer comunicação com o nome *localhost* acabará sendo tratada pelo sistema local.

In [ ]:
%%writefile hosts

localhost slots=4

# O arquivo de hosts é um arquivo texto, contendo, em cada linha, a especificação
# de um computador onde é possível iniciar a execução das aplicações.
#
# no1.domain.com
# no2.domain.com

# A ordem dos hosts listados pode não ser seguida na ativação do programa.
# Para cada host, é possível especificar também o número de "slots", que representa
# o número de instâncias (processos) que podem ser ativadas neste nó.
# Normalmente, esse número é relacionado ao número de processadores disponíveis no nó.
#
# no1 slots=2
# no2 slots=2
# no3 slots=4

# O número máximo de slots também pode ser especificado e indica o número
# máximo de processos que podem ser atribuídos ao nó.
#
# no1 slots=4 max-slots=4

# Uma vez definidos os computadores disponíveis no arquivo de hosts, a indicação
# de quais computadores usar pode ser feita especificando o nome do arquivo na
# ativação do programa
#
# $ mpirun -np 4 --hostfile hosts prog
# ou
# $ mpiru -np 4 --machinefile hosts prog

Overwriting hosts


O exemplo de execução a seguir tem alguns comandos que permitem obter diferentes informações sobre o *hostname* e sobre a versão MPI disponível.

In [ ]:
! lscpu | grep CPU && echo
! echo "hostname: " $(hostname) && echo
! which mpicc mpirun orterun && echo
! mpirun --version

CPU op-mode(s):                     32-bit, 64-bit
CPU(s):                             2
On-line CPU(s) list:                0,1
Model name:                         Intel(R) Xeon(R) CPU @ 2.20GHz
CPU family:                         6
NUMA node0 CPU(s):                  0,1

hostname:  088273c1df5b

/usr/bin/mpicc
/usr/bin/mpirun
/usr/bin/orterun

mpirun (Open MPI) 4.1.2

Report bugs to http://www.open-mpi.org/community/help/


Os comandos a seguir realizam a compilação do programa m1.c, apresentado anteriormente, e o exeutam usando 4 instâncias de processos no host local.

In [ ]:
# ! mpicc m1.c -o m1 && mpirun --allow-run-as-root -np 4 -host localhost,localhost,localhost,localhost m1
# ! mpicc m1.c -o m1 && mpirun --allow-run-as-root -np 4 -host localhost:4 m1
! mpirun --allow-run-as-root -np 4 -hostfile hosts m1

Vejam que mesmo o programa estando compilado, é possível iniciar sua execução usando diferentes números e conjuntos de computadores!

In [ ]:
! if [ ! m1 -nt m1.c ]; then mpicc m1.c -o m1; fi
! mpirun --allow-run-as-root -host localhost:6 -np 6 m1

Processo 3 de 6 em d13eb2bab234
Processo 5 de 6 em d13eb2bab234
rank (5): 0: m1    
Processo 0 de 6 em d13eb2bab234
Processo 4 de 6 em d13eb2bab234
rank (4): 0: m1    
rank (3): 0: m1    
rank (0): 0: m1    
Processo 1 de 6 em d13eb2bab234
rank (1): 0: m1    
Processo 2 de 6 em d13eb2bab234
rank (2): 0: m1    


# Criando programas MPI: modelo de processos

Na programação paralela distribuída, ou seja, usando passagem de mensagens para as comunicações entre as tarefas, é comum usar-se um modelo **mestre / trabalhador** (*master / worker*).

Em geral, cabe ao processo **mestre** **enviar os dados** para os processos trabalhadores, que irão manipulá-los, e aguardar os resultados. Já nos processos que estiverem fazendo o papel de **trabalhador**, é preciso esperar pelos dados, processá-los localmente e enviar os resultados de volta ao servidor.

Esse modelo é comum quando os dados a manipular estão em arquivo, acessível por um nó apenas. Cabe então ao processo nesse nó ler os dados e repassá-los, **em partes**, aos demais.

    Obs 1: Um aspecto importante a observar aqui é que estamos tratando de **processos**,
    já que trata-se de uma aplicação que envolve atividades em diferentes nós. Ou seja,
    diferentes sistemas computacionais conectados em rede. É claro que cada processo
    poderá ter várias threads, mas trataremos disso posteriormente...

    Obs 2: É claro que é possível ter um sistema de arquivos distribuídos,
    de forma que cada processo leia os dados localmente. Isso é feito em
    alguns cenários, como com MPI_IO, por exemplo.

```
    Master      W1      W2     W3     W4   ...  Wn
      |          |      |      |      |         |
      | -------->|      |      |      |   ...   |   W1 recebe e começa processar
      | ---------|----->|      |      |   ...   |   W2 recebe e começa ...
      | ---------|------|----->|      |         |    ...
      | ---------|------|------|----->|         |    ...
      |   ...    |      |      |      |         |    ...
      | ---------|------|------|------|-------->|   Wn recebe e começa ...
      |    .     |      |      |      |         |
      |    .     |      |      |      |         |
      |    .     |      |      |      |         |
      |    .     |      |      |      |         |
      |<---------|      |      |      |   ...   |  W1 conclui e envia resultados
      |<---------|------|      |      |   ...   |  W2 envia resultados
      |<---------|------|------|      |         |   ...
      |<---------|------|------|------|         |   ...
      |   ...    |      |      |      |         |
      |<---------|------|------|------|---------|  Wn envia resulados
      |          |      |      |      |         |

```

Há situações em que há várias "rodadas" dessa forma de interação no programa. Há casos também de processamento contínuo, em que o servidor e os clientes ficam permanentemente nesse ciclo de envio e recebimento.

Também é possível usar um modelo de ***pipeline***, em que cada processo faz uma manipulação nos dados de entrada e repassa os dados processados para um próximo nó. Esse próximo nó repassa os dados para o próximo e aguarda novos dados. Tudo isso num ciclo.

```
    Master      W1      W2     W3     W4   ...  Wn
      |         |      |      |      |         |
      |-------->|      |      |      |   ...   |  W1 recebe e começa processar
      |         |      |      |      |         |
      |         |      |      |      |         |
      |         |----->|      |      |   ...   |  W2 recebe e começa ...
      |-------->|      |      |      |         |
      |         |      |      |      |         |
      |         |      |----->|      |         |  W3 recebe e começa ...
      |         |----->|      |      |         |
      |-------->|      |      |      |    ...
      |         |      |      |----->|         |
      |         |      |----->|      |         |
      |         |----->|      |      |         |
      |-------->|      |      |      |         |
      |         |      |      |      |         |
      |         |      |      |      |---...-->|  Wn recebe e começa...
      |   ...   |      |      |----->|         |  ... pipeline entra em    
      |         |      |----->|      |         |      regime de produção,
      |         |----->|      |      |         |      com todos os nós
      |-------->|      |      |      |         |      trabalhando!
      |         |      |      |      |         |
      |   ...   |      |      |      |         |
```

Como veremos posteriormente, MPI também possui primitivas para comunicação coletiva, que permitem o envio de mensages para todos os processos em um grupo.





# Criando programa MPI: SPMD x MPMD

Seja qual for o modelo de processos adotado na programação, *master/worker*, *pipeline*, ou outro, cabe ainda ao programador decidir se ele vai criar todo o código no mesmo programa ou se vai usar programas separados para cada papel.

Nos casos mais comuns, a criação de programas com MPI é feita no modelo SPMD. Isso quer dizer que o programador cria um único programa, que vai ser iniciado em todos os nós de processamento.

Assim, é usual criar os 2 (ou mais) papéis dentro do mesmo código e usar o ***rank*** do processo para determinar qual parte do código será executada.

Neste caso, compila-se este programa e pode-se submetê-lo à execução usando diferentes números de processos e nós de processamento.

No início do código, é comum que o programa contenha chamadas às funções MPI para determinar quantos processos estão sendo usados nesta execução e dividir as atividades de maneira apropriada. Tem-se, assim um programa que pode adequar-se à escalabilidade do sistema computacional.

Desta forma, fica bem mais fácil para o programador experimentar executar o programa com números variados de processos e determinar qual é a configuração que resulta em melhor desempenho.

## Iniciando programas MPI no modelo SMPD

Usando um código só, que define a função de cada processo internamente, a ativação de um programa MPI fica pareceida com algo assim:

```
$ mpirun prog -n XXX -hostfile HHH ...
```

Já se forem criadas versões separadas para o código do *mestre* e o código dos *trabalhadores*, a linha de comando fica parecida com:

```
$ mpirun master  -n XXX  worker  -hostfile HHH ...
```



In [ ]:
%%writefile m-w.c

#include <mpi.h>
#include <stdio.h>

int
main( int argc, char *argv[])
{
	int numtasks, rank, status;

	status = MPI_Init(&argc,&argv);

	if (status != MPI_SUCCESS) {
		printf ("Erro em MPI_Init. Terminando...\n");
		MPI_Abort(MPI_COMM_WORLD, status);
	}
  // obtém o número de processos sendo usados nesta execução
	MPI_Comm_size(MPI_COMM_WORLD,&numtasks);

  // obtém o rank deste processo em relação aos processos da aplicação
	MPI_Comm_rank(MPI_COMM_WORLD,&rank);

  if(rank==0) {
    printf("Master (%d / %d)\n",rank,numtasks);
  } else {
     printf("Worker (%d / %d)\n",rank,numtasks);
  }

	// talvez devesse esperar os processos filhos terminarem antes de terminar...

	MPI_Finalize();

	return(0);
}

Overwriting m-w.c


In [ ]:
%%writefile master.c

#include <mpi.h>
#include <stdio.h>
#include <unistd.h>

int
main( int argc, char *argv[])
{
	int numtasks, rank, status;

	status = MPI_Init(&argc,&argv);

	if (status != MPI_SUCCESS) {
		printf ("Erro em MPI_Init. Terminando...\n");
		MPI_Abort(MPI_COMM_WORLD, status);
	}
  // obtém o número de processos sendo usados nesta execução
	MPI_Comm_size(MPI_COMM_WORLD,&numtasks);

  // obtém o rank deste processo em relação aos processos da aplicação
	MPI_Comm_rank(MPI_COMM_WORLD,&rank);

  // Código executado só pelo processo master
  printf("Master (%d / %d)\n",rank,numtasks);

	// Na prática, deveria esperar os demais processos terminarem antes de encerrar a aplicação...
	sleep(3);

	MPI_Finalize();

	return(0);
}

Writing master.c


In [ ]:
%%writefile worker.c

#include <mpi.h>
#include <stdio.h>

int
main( int argc, char *argv[])
{
	int numtasks, rank, status;
	char processor_name[MPI_MAX_PROCESSOR_NAME];

	status = MPI_Init(&argc,&argv);

	if (status != MPI_SUCCESS) {
		printf ("Erro em MPI_Init. Terminando...\n");
		MPI_Abort(MPI_COMM_WORLD, status);
	}
  // obtém o número de processos sendo usados nesta execução
	MPI_Comm_size(MPI_COMM_WORLD,&numtasks);

  // obtém o rank deste processo em relação aos processos da aplicação
	MPI_Comm_rank(MPI_COMM_WORLD,&rank);

  // Código executado só pelos processos workers
  printf("Worker (%d / %d)\n",rank,numtasks);

	MPI_Finalize();

	return(0);
}

Overwriting worker.c


In [ ]:
! echo "Executando um programa no modelo SPMD (mesmo programa no master e nos workers)"
! if [ ! m-w -nt m-w.c ]; then mpicc m-w.c -o m-w; fi
! mpirun --allow-run-as-root -np 4 -host localhost:4 m-w

Executando um programa no modelo SPMD (mesmo programa no master e nos workers)
Worker (3 / 4)
Worker (2 / 4)
Worker (1 / 4)
Master (0 / 4)


In [ ]:
! echo "Executando um programa no modelo MPMD (programas distintos no master e nos workers)"
! if [ ! master -nt master.c ]; then mpicc master.c -o master; fi
! if [ ! worker -nt worker.c ]; then mpicc worker.c -o worker; fi
# ! mpirun --allow-run-as-root -np 1 -host localhost:5 master : -np 4 worker -host localhost:5
! mpirun --allow-run-as-root -np 1 master : -np 4 -host localhost:5 worker

Executando um programa no modelo MPMD (programas distintos no master e nos workers)
Worker (2 / 5)
Worker (3 / 5)
Worker (1 / 5)
Worker (4 / 5)
Master (0 / 5)
